In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_excel("./process_data/dining area.xlsx")
df.head()

,Unnamed: 0,start_time,end_time,temperature,luminosity,humidity,motion
0,0,2022-05-01 00:00:00,2022-05-01 01:30:00,22.031250,0.0,46.61,0
1,1,2022-05-01 01:30:00,2022-05-01 03:00:00,21.865000,0.0,48.00,1
2,2,2022-05-01 03:00:00,2022-05-01 04:30:00,21.443333,3.0,48.50,1
3,3,2022-05-01 04:30:00,2022-05-01 06:00:00,21.360000,69.0,49.00,0
4,4,2022-05-01 06:00:00,2022-05-01 07:30:00,20.992500,94.0,48.00,0


In [4]:
df["temperature"].describe()

count    7010.000000
mean       23.561677
std         3.609479
min         4.825109
25%        20.421250
50%        23.367222
75%        26.501500
max        35.401667
Name: temperature, dtype: float64

In [5]:
df["humidity"].describe()

count    7010.000000
mean       52.576881
std        11.712454
min         6.936949
25%        44.000000
50%        51.000000
75%        61.000000
max        81.500000
Name: humidity, dtype: float64

# 1. Data preprocessing

- Chia dữ liệu intervals thành dữ liệu time-series. Có 2 khoảng thời gian chính bao gồm:
    - `4:30`: `12:00`
    - `12:30`: `22:30`

- Normalization

## 1.1. Data segmentation

Chia dữ liệu thành:
- `4:30` -> `12:00`
- `12:30` -> `22:30`

In [34]:
df = df.drop(columns = ['Unnamed: 0'])
df.head()

,start_time,end_time,temperature,luminosity,humidity,motion
0,2022-05-01 00:00:00,2022-05-01 01:30:00,22.031250,0.0,46.61,0
1,2022-05-01 01:30:00,2022-05-01 03:00:00,21.865000,0.0,48.00,1
2,2022-05-01 03:00:00,2022-05-01 04:30:00,21.443333,3.0,48.50,1
3,2022-05-01 04:30:00,2022-05-01 06:00:00,21.360000,69.0,49.00,0
4,2022-05-01 06:00:00,2022-05-01 07:30:00,20.992500,94.0,48.00,0


In [35]:
morning_start = pd.to_datetime("04:30:00").time()
morning_end = pd.to_datetime("12:00:00").time()

afternoon_start = pd.to_datetime("12:00:00").time()
afternoon_end = pd.to_datetime("20:30:00").time()

segment_data = {
    "time": [],
    "temperature": [],
    "luminosity": [],
    "humidity": [],
    "motion": []
}

In [37]:
dates = df["start_time"].dt.date.unique()
datatypes = [col for col in df.columns if col not in {"start_time", "end_time"}]

df_data = {
    "date": [],
    "time": [],
    "temperature": [],
    "luminosity": [],
    "humidity": [],
    "motion": []
}

datatypes

['temperature', 'luminosity', 'humidity', 'motion']

In [41]:
for date in dates:
    rows = df[
        df["start_time"].dt.date == date
    ]

    """---------------MORNING TIME---------------"""
    morning_df = rows[
        (rows["start_time"].dt.time >= morning_start) &
        (rows["start_time"].dt.time <= morning_end) &
        (rows["end_time"].dt.time <= morning_end)
    ].sort_values("start_time")

    for datatype in datatypes:
        df_data[datatype].append(morning_df[datatype].tolist())

    df_data["date"].append(date)
    df_data["time"].append(f"{morning_start} - {morning_end}")

    """---------------AFTERNOON TIME--------------"""
    afternoon_df = rows[
        (rows["start_time"].dt.time >= afternoon_start) &
        (rows["start_time"].dt.time <= afternoon_end) &
        (rows["end_time"].dt.time <= afternoon_end)
    ]

    for datatype in datatypes:
        df_data[datatype].append(afternoon_df[datatype].tolist())

    df_data["date"].append(date)
    df_data["time"].append(f"{afternoon_start} - {afternoon_end}")

In [42]:
time_df = pd.DataFrame(df_data)
time_df.head(10)

,date,time,temperature,luminosity,humidity,motion
0,2022-05-01,04:30:00 - 12:00:00,"[21.36, 20.9925, 20.82333333333333, 21.67, 23....","[69.0, 94.0, 141.0, 406.49, 334.0]","[49.0, 48.0, 47.0, 45.0, 42.5]","[0, 0, 2, 4, 7]"
1,2022-05-01,12:00:00 - 20:30:00,"[25.4266665, 23.56583333333333, 23.6625, 23.22...","[138.0, 105.0, 80.0, 42.75, 0.0]","[35.0, 34.0, 34.0, 35.75, 42.0]","[3, 3, 1, 1, 6]"
2,2022-05-02,04:30:00 - 12:00:00,"[21.055111, 21.12083333333333, 21.092222333333...","[48.0, 101.0, 365.5, 529.0, 962.0]","[47.0, 51.5, 46.5, 41.0, 38.0]","[1, 4, 2, 0, 1]"
3,2022-05-02,12:00:00 - 20:30:00,"[24.78, 24.42, 23.91541675, 23.50125, 23.01583...","[1107.0, 120.5, 64.5, 52.0, 9.5]","[35.0, 34.0, 34.0, 34.0, 37.5]","[0, 0, 5, 6, 5]"
4,2022-05-03,04:30:00 - 12:00:00,"[21.495, 21.515, 21.49, 21.99444433333333, 23....","[82.0, 91.0, 151.0, 400.5, 591.0]","[52.0, 50.0, 49.0, 47.5, 40.5]","[5, 1, 1, 4, 4]"
5,2022-05-03,12:00:00 - 20:30:00,"[23.998889, 24.16, 23.78, 23.15416675, 22.595]","[634.0, 147.0, 42.0, 22.0, 3.5]","[39.0, 39.0, 41.0, 45.0, 51.0]","[0, 0, 3, 1, 4]"
6,2022-05-04,04:30:00 - 12:00:00,"[21.0525, 21.12, 21.17777766666667, 22.16375, ...","[14.0, 119.5, 174.0, 763.5, 511.0]","[55.0, 54.0, 55.0, 48.5, 42.0]","[2, 4, 5, 2, 6]"
7,2022-05-04,12:00:00 - 20:30:00,"[24.2875, 23.8583335, 23.56, 22.977778, 22.813...","[661.0, 185.5, 28.0, 5.0, 7.0]","[38.5, 38.5, 40.0, 43.0, 45.0]","[1, 3, 1, 3, 5]"
8,2022-05-05,04:30:00 - 12:00:00,"[21.14, 21.18, 21.22375, 21.294375, 21.45]","[8.5, 35.0, 34.0, 145.0, 79.0]","[55.0, 61.0, 59.0, 57.5, 57.0]","[4, 4, 3, 6, 3]"
9,2022-05-05,12:00:00 - 20:30:00,"[21.4, 21.36, 21.07916666666667, 21.218889, 21...","[111.0, 101.0, 155.0, 22.5, 7.0]","[56.0, 56.0, 55.5, 55.5, 58.0]","[0, 2, 3, 3, 1]"
